In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from utils_accelerate import *

tokenizer = T5Tokenizer.from_pretrained('t5-small')
# input = "predict tail: barack obama | position_held |"
# input = "translate English to German: How are you doing?"

# model = T5ForConditionalGeneration.from_pretrained('models/codex_m_accelerate_1gpu.pt')
checkpoint_location = 'models/codex_m_accelerate_1gpu/115000.pt'
# checkpoint_location = 'models/codex-m_tiny/70000.pt'
# checkpoint_location = 'models/codex-m_small_6gpu/45000.pt'
model = load_accelerator_model(checkpoint_location, only_model=True)
model.eval()
model.cpu()

Model args
Namespace(batch_size=64, dataset='codex-m', epochs=20, learning_rate=None, load_checkpoint=None, loss_steps=500, max_checkpoints=5, model_size='small', num_workers=3, optimizer='adafactor', resume=None, save_prefix='codex_m_accelerate_1gpu', save_steps=5000, start_steps=0)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [2]:
input = "predict tail: united states of america | member of |"
input_ids = tokenizer(input, return_tensors="pt").input_ids  # Batch size 1
model.cpu()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [3]:
# outputs = model.sample(input_ids)
from transformers import (
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    BeamSearchScorer,
)

In [4]:
fname = 'data/codex-m/test.txt'
f = open(fname, 'r')
data = []
for line in f:
    data.append(line.strip())
f.close()

In [5]:
len(data)

20622

In [6]:
data[0]

'predict tail: billy idol | languages spoken, written, or signed |\tenglish'

In [7]:
import torch
# data_point = 'predict tail: novalis | occupation |    philosopher'
id = 0
data_point = data[id]
encoder_input_str, target = data_point.split('\t')
encoder_input_str = [encoder_input_str]
encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
num_beams = 10
num_predictions = 3
input_ids = torch.ones((len(encoder_input_str) * num_beams, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.decoder_start_token_id
model_kwargs = {
    "encoder_outputs": model.get_encoder()(encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True)
}
beam_scorer = BeamSearchScorer(
    batch_size=len(encoder_input_str),
    max_length=model.config.max_length,
    num_beams=num_beams,
    device=model.device,
    num_beam_hyps_to_keep=num_predictions,
    length_penalty=0.3
)
logits_processor = LogitsProcessorList([])

In [8]:
encoder_input_str

['predict tail: billy idol | languages spoken, written, or signed |']

In [9]:
outputs = model.beam_search(input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs)
print("Beam:", tokenizer.batch_decode(outputs, skip_special_tokens=True))
outputs = model.generate(encoder_input_ids)
print('Greedy:', tokenizer.batch_decode(outputs, skip_special_tokens=True))
print('Target:', target)

Beam: ['english', 'german', 'british english']
Greedy: ['english']
Target: english


In [10]:
def getGreedyOutput(model, tokenizer, encoder_input_str):
    encoder_input_str = [encoder_input_str]
    encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
    outputs = model.generate(encoder_input_ids)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [31]:
def getBeamOutput(model, tokenizer, encoder_input_str, num_beams=10, 
                  num_predictions=3, length_penalty=0.3):
    encoder_input_str = [encoder_input_str]
    encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
#     encoder_input_ids = tokenizer(encoder_input_str, padding='max_length', truncation=True, max_length=128, return_tensors="pt").input_ids
    input_ids = torch.ones((len(encoder_input_str) * num_beams, 1), device=model.device, dtype=torch.long)
    input_ids = input_ids * model.config.decoder_start_token_id
    model_kwargs = {
        "encoder_outputs": model.get_encoder()(encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True)
    }
    beam_scorer = BeamSearchScorer(
        batch_size=len(encoder_input_str),
        max_length=model.config.max_length,
        num_beams=num_beams,
        device=model.device,
        num_beam_hyps_to_keep=num_predictions,
        length_penalty=length_penalty
    )
    logits_processor = LogitsProcessorList([])
    outputs = model.beam_search(input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [26]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [32]:
from tqdm import tqdm
# id = 100
scorer_function = getBeamOutput 
# scorer_function = getGreedyOutput 
num_points = 50
correct = 0
for id in tqdm(range(0, num_points)):
    data_point = data[id]
    input, target = data_point.split('\t')
    predicted = set(scorer_function(model, tokenizer, input))
    print(predicted, target)
    if target in predicted:
        correct += 1
print(correct/num_points)

  2%|▏         | 1/50 [00:00<00:15,  3.10it/s]

{'english', 'british english', 'german'} english


  4%|▍         | 2/50 [00:00<00:17,  2.75it/s]

{'edgar allan poe', 'samuel r. delany', 'edward bernard raczynski'} billy idol


  6%|▌         | 3/50 [00:01<00:15,  3.09it/s]

...

{'robert moog', 'joel mcneely', 'james cotton'} emily osment


 98%|█████████▊| 49/50 [00:15<00:00,  2.70it/s]

{'leipzig', 'berlin', 'heidelberg'} berlin


100%|██████████| 50/50 [00:16<00:00,  3.08it/s]

{'johann erich biester', 'wilhelm liebknecht', 'ernst niekisch'} theodor fontane
0.26


In [118]:
outputs.shape

torch.Size([20, 15])

In [71]:
outputs

tensor([[    0,  1038,  2137,    21, 20532,    11,   606,     1]])

Generated: ['international bank for reconstruction and development']


In [3]:
print(input)
print(''.join(tokenizer.convert_ids_to_tokens(outputs[0])))

predict tail: united states of america | member of |
<pad>▁international▁bank▁for▁reconstruction▁and▁development</s>


In [4]:
from dataset import T5_Dataset

In [5]:
valid_dataset = T5_Dataset('test', dataset_name='codex-m')

100%|██████████| 20622/20622 [00:00<00:00, 840393.08it/s]


In [6]:
from eval_accelerate import removePadding, eval

In [7]:
class Args:
    batch_size = 200
args=Args()

In [8]:
acc = eval(model, valid_dataset, args)

100%|██████████| 104/104 [00:59<00:00,  1.74batches/s]


In [9]:
acc

0.10876733585491223

In [8]:
actual = tokenizer("international development association", return_tensors="pt").input_ids[0].numpy()

In [9]:
actual

array([1038,  606, 6028,    1])

In [10]:
predicted = outputs[0][1:].numpy()

In [11]:
predicted

array([ 1038,  2137,    21, 20532,    11,   606,     1])

In [30]:
actual == predicted

array([ True,  True,  True,  True])

In [25]:
actual.numpy()

array([1038,  606, 6028,    1])